<a href="https://colab.research.google.com/github/harry-graves/Aria_ORI/blob/main/Aria_Calibration_Parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Running from Google Colab, installing projectaria_tools")
!pip install projectaria-tools

In [5]:
from projectaria_tools.core import data_provider, calibration
from projectaria_tools.core.image import InterpolationMethod
from projectaria_tools.core.sensor_data import TimeDomain, TimeQueryOptions
from projectaria_tools.core.stream_id import RecordableTypeId, StreamId
import numpy as np
import os
from matplotlib import pyplot as plt
from google.colab import drive

In [18]:
if not os.path.exists('Calibration.vrs'):
    print('Please upload VRS file to Colab workspace!')
else:
  vrsfile = 'Calibration.vrs'

In [ ]:
print(f"Creating data provider from {vrsfile}")
provider = data_provider.create_vrs_data_provider(vrsfile)
if not provider:
    print("Invalid vrs data provider")

In [ ]:
device_calib = provider.get_device_calibration()
all_sensor_labels = device_calib.get_all_labels()
print(f"device calibration contains calibrations for the following sensors \n {all_sensor_labels}")

In [ ]:
camera_name = "camera-rgb"
transform_device_camera = device_calib.get_transform_device_sensor(camera_name).to_matrix()
transform_camera_device = np.linalg.inv(transform_device_camera)
print(f"Device calibration origin label {device_calib.get_origin_label()}")
print(f"{camera_name} has extrinsics of \n {transform_device_camera}")

rgb_calib = device_calib.get_camera_calib("camera-rgb")

In [ ]:
et_calib = device_calib.get_aria_et_camera_calib()
if et_calib is not None:
    print(f"Camera {et_calib[0].get_label()} has image size {et_calib[0].get_image_size()}")
    print(f"Camera {et_calib[1].get_label()} has image size {et_calib[1].get_image_size()}")

imu_calib = device_calib.get_imu_calib("imu-left")
if imu_calib is not None:
    print(f"{imu_calib.get_label()} has extrinsics transform_Device_Imu:\n {imu_calib.get_transform_device_imu().to_matrix3x4()}")

In [ ]:
# Retrieve device calibration
device_calib = provider.get_device_calibration()

# Define the sensors to retrieve data for
sensors = ["imu-left", "imu-right", "camera-slam-left", "camera-slam-right", "camera-rgb"]

# Iterate over each sensor and print the required information
for sensor in sensors:
    if "imu" in sensor:
        # Retrieve and print IMU extrinsics
        imu_calib = device_calib.get_imu_calib(sensor)
        if imu_calib is not None:
            print(f"{imu_calib.get_label()} has extrinsics transform_Device_Imu:\n{imu_calib.get_transform_device_imu().to_matrix3x4()}\n")
    else:
        # Retrieve and print camera extrinsics and intrinsics
        camera_calib = device_calib.get_camera_calib(sensor)
        if camera_calib is not None:
            extrinsics = device_calib.get_transform_device_sensor(sensor).to_matrix()
            focal_lengths = camera_calib.get_focal_lengths()
            print(f"{sensor} has extrinsics of \n{extrinsics}")
            print(f"{sensor} has focal lengths: {focal_lengths}\n")